# Task: Build a simple pipeline with DVC

In this task we want to build a simple DVC pipeline. The elements of the pipeline will classify images into either *lemons* or *bananas*.

The pipelines consists of 2 python functions and one shell command for deployment:

1. `preprocess(inputpath, outputpath)`, that processes images (convert to grayscale, resize to (100, 100)). `inputpath` is the location of the input image, and `outputpath` the location of the preprocessed image. 
2. `classify(inputpath, outputpath)`, that classifies images and write the results into a JSON file. `inputpath` is the location of the preprocessed image, and `outputpath` the location of the JSON file.
3. `"cat deployment_location | xargs -I% cp jsonfile %"`. `deployment_location` is a file containing the location we want to deploy to, and `jsonfile` should be the location of the JSON file.

We have already provided the python functions for you. You can find them in the file `DVC_exercise.py`. Your mission is to wrap a pipeline around them using DVC.

You can call the functions from the shell via `python <file> <function name> <input params>`.

Remember that you can run shell commands in a Jupyter notebook by starting a line in a cell with `!` or putting `%%sh` at the beginning of a cell.

#### Initialise DVC

Please use `--no-scm` to avoid problems with this git repository.

In [64]:
!dvc init -f --no-scm

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|              https://dvc.org/doc/user-guide/analytics               |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: https://dvc.org/doc
- Get help and share ideas: https://dvc.org/chat
- Star us on GitHub: https://github.com/iterative/dvc


#### Define first pipeline stage

In [65]:
!dvc run -f preprocess.dvc -d DVC_exercise.py -d exercise-dataset-dvc/image.jpg -o preprocessed.jpg \
         python DVC_exercise.py preprocess exercise-dataset-dvc/image.jpg preprocessed.jpg

Running command:
	python DVC_exercise.py preprocess exercise-dataset-dvc/image.jpg preprocessed.jpg
Saving 'preprocessed.jpg' to '.dvc/cache/9b/e73f582255331b84c3de936f6cc8f8'.
Saving information to 'preprocess.dvc'.


#### Define second pipeline stage

In [66]:
!dvc run -f classify.dvc -d DVC_exercise.py -d preprocessed.jpg -o result.json \
         python DVC_exercise.py classify preprocessed.jpg result.json

Running command:
	python DVC_exercise.py classify preprocessed.jpg result.json
Saving 'result.json' to '.dvc/cache/c7/d4ca78a77357d89789d675ea96d969'.
Saving information to 'classify.dvc'.


#### Define third pipeline stage
Tip: Don't specify the name of the .dvc file in the last stage. DVC will then use the default name DVCfile, which is handy for later reproduction.

In [69]:
!dvc run -d result.json -d deployment_location \
         "cat deployment_location | xargs -I% cp result.json %"

Running command:
	cat deployment_location | xargs -I% cp result.json %
Saving information to 'Dvcfile'.


#### Check your pipeline here:

In [70]:
%%sh
dvc pipeline show --ascii

+----------------+ 
| preprocess.dvc | 
+----------------+ 
         *         
         *         
         *         
 +--------------+  
 | classify.dvc |  
 +--------------+  
         *         
         *         
         *         
    +---------+    
    | Dvcfile |    
    +---------+    


#### Modify the deployment location and reproduce the pipeline
We want to change our pipeline slightly. Our customer would like the classified fruit to be deployed (copied) to `./output-exercise` now.

In [74]:
# change desired deployment location
!echo output-exercise > deployment_location

In [73]:
!dvc repro

Stage 'preprocess.dvc' didn't change.
Stage 'classify.dvc' didn't change.
Reproducing 'Dvcfile'
Running command:
	cat deployment_location | xargs -I% cp result.json %
Saving information to 'Dvcfile'.


In [79]:
# debug code
!rm -rf *.dvc